# 0. Magic

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [0]:
import torch
from torch import tensor
from torch.nn import *
import torch.nn.functional as F
from torch.utils import data 
from fastai import datasets
import pickle, gzip, math, torch

# 2. Data

In [0]:
class Dataset(data.Dataset):
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [0]:
x_train, y_train, x_valid, y_valid = get_data()
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(valid_ds, bs)

In [0]:
nh, bs = 100, 32
c = y_train.max() + 1
loss_func = F.cross_entropy

# 3. DataBunch

สร้าง Class DataBunch สำหรับ Wrap Training Set DataLoader และ Validation Set DataLoader 

และใส่ จำนวน Class ของ Output ไว้ด้วย จะได้ใช้ตอนสร้างโมเดล

In [0]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

ลองสร้าง DataBunch จาก train_dl, valid_dl และ c ที่เราสร้างไว้ก่อนหน้านี้

In [0]:
databunch = DataBunch(train_dl, valid_dl, c)

เราจะได้ Training Loop ที่โค้ด Clean มากขึ้น

In [0]:
fit(epoch, model, opt, databunch)

# Credit

* https://course.fast.ai/videos/?lesson=9